# Subway Entrance to Census block mapping
This code maps the subway entrances across NYC to census blocks groups. The output is essentially a .csv file with two columns spatial id (of census block) and number of subway entrances therein

In [ ]:
from datetime import datetime, timedelta
import geopandas as gpd
import json
import pandas as pd
import mapclassify
import matplotlib.pyplot as plt
import numpy as np
import os
import requests
from io import StringIO
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)
np.set_printoptions(threshold=np.inf)

In [ ]:
# Subway Entrances
subways_url = 'https://data.cityofnewyork.us/resource/he7q-3hwy.geojson'

In [ ]:
subways_response = requests.get(subways_url)
subways_json = subways_response.json()
subways_gdf = gpd.GeoDataFrame.from_features(subways_json['features'])
subways_gdf = subways_gdf[['geometry','objectid']]
print(subways_gdf)

In [ ]:
subways_gdf.plot(figsize=(15,15))

In [ ]:
parent_dir = os.path.abspath('..')  # get the absolute path of the parent directory
file_path = os.path.join(parent_dir, 'Data', 'nyc_bgrp.geojson')  # construct the file path
cb_gdf = gpd.read_file(file_path)  # load the GeoJSON file into a GeoDataFrame
cb_gdf.head(2)
cb_gdf.plot(figsize=(30,30))
print(len(cb_gdf))

In [ ]:
subways_joined_tracts_gdf = gpd.sjoin(subways_gdf, cb_gdf[['geometry','spatial_id']], op='within', how='left')

In [ ]:
print(subways_joined_tracts_gdf)

In [ ]:
# Group by census block and count the number of subway entrances
subway_counts = subways_joined_tracts_gdf.groupby('spatial_id').size().reset_index(name='subway_count')

In [ ]:
# Merge subway counts with census blocks data
subway_census_blocks_df = cb_gdf.merge(subway_counts, on='spatial_id', how='left')
subway_census_blocks_df['subway_count'].fillna(0,inplace =True)
print(subway_census_blocks_df)

In [ ]:

# Plot the census blocks with subway entrance counts
fig, ax = plt.subplots(figsize=(12, 8))
subway_census_blocks_df.plot(column='subway_count', cmap='coolwarm', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True, vmin=0, vmax=subway_census_blocks_df['subway_count'].max())
ax.set_title('Number of Subway Entrances in each Census Block in NYC')
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap
max_count = int(subway_census_blocks_df['subway_count'].max())

# Define a custom colormap with blue for 0 subway entrances and red for higher entrance counts
cmap = ListedColormap(['white'] + ['black'] * max_count)

# Plot the census blocks with subway entrance counts using the custom colormap
fig, ax = plt.subplots(figsize=(12, 8))
subway_census_blocks_df.plot(column='subway_count', cmap=cmap, linewidth=0.1, ax=ax, edgecolor='0.8', legend=True, vmin=0, vmax=max_count)
ax.set_title('Number of Subway Entrances in each Census Block in NYC')
plt.show()

In [ ]:
export_columns = ['spatial_id'] + ['subway_count'] 
print(export_columns)
parent_dir = os.path.abspath('..')  # get the absolute path of the parent directory
file_path = os.path.join(parent_dir, 'Data', 'subway_entrances_2_censusblocks.csv')  # construct the file path
subway_census_blocks_df[export_columns].to_csv(file_path, index=False)